In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from datetime import datetime
from collections import Counter

In [27]:
df = pd.read_csv('data/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')
print(df.shape)
df.head()

(2215024, 13)


,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,150060275,NON-CRIMINAL,LOST PROPERTY,Monday,01/19/2015,14:00,MISSION,NONE,18TH ST / VALENCIA ST,-122.421582,37.761701,"(37.7617007179518, -122.42158168137)",15006027571000
1,150098210,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821003074
2,150098210,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821004014
3,150098210,SECONDARY CODES,DOMESTIC VIOLENCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821015200
4,150098226,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,01/27/2015,19:00,NORTHERN,NONE,LOMBARD ST / LAGUNA ST,-122.431119,37.800469,"(37.8004687042875, -122.431118543788)",15009822628160


## Sanity check

### 1. Inspect data for any duplicates

To-dos:
* To calculate total number of crimes, make sure not to double count
* Double counting is alright if calculating based on crime types but delete if both incident_num and category are the same

From the metadata, we understand that PdId refers to the unique identifier for use in update and insert operations for the dataset, hence we can drop the column.

In [28]:
df=df.drop(['PdId'], axis=1)

In [29]:
df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
0,150060275,NON-CRIMINAL,LOST PROPERTY,Monday,01/19/2015,14:00,MISSION,NONE,18TH ST / VALENCIA ST,-122.421582,37.761701,"(37.7617007179518, -122.42158168137)"
1,150098210,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)"
2,150098210,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)"
3,150098210,SECONDARY CODES,DOMESTIC VIOLENCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)"
4,150098226,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,01/27/2015,19:00,NORTHERN,NONE,LOMBARD ST / LAGUNA ST,-122.431119,37.800469,"(37.8004687042875, -122.431118543788)"


In [30]:
# check unique no. of rows (corresponding to no. of incident numbers)
print("No. of rows:",df.shape[0])

No. of rows: 2215024


In [31]:
# check unique no. of incidents
print("No. of unique incidents:", df['IncidntNum'].nunique())

No. of unique incidents: 1746914


In [32]:
df.shape[0]-df['IncidntNum'].nunique()

468110

Hence we can conclude that there were 468110 duplicates in incident_num. Let's take a look at an example of a duplicate incident. The logic is that, if the duplicates arise due to the incident being categorized into various incidents of crime, we can retain the double-counting; however, if the duplicates are merely a result of same incident_num and category, we should delete these rows.

In [33]:
incidentNumList=Counter(df['IncidntNum']) # obtain dictionary of all unique key-value pairs
# print(incidentNumList)

In [34]:
allIncidentNumbers=df['IncidntNum'].unique().tolist()

In [35]:
len(incidentNumList)

1746914

In [36]:
threshold=1
duplicateIncidentList=[]
isolatedIncidentList=[]
for incident, incidentCount in incidentNumList.items():
    if incidentCount>threshold:
        duplicateIncidentList.append(incident)
    elif incidentCount==threshold:
        isolatedIncidentList.append(incident)

# shorter code is this: 
# isolatedIncidents=[incd for incd, incidentCount in incidentNumList.items() if incidentCount == threshold]
# duplicateIncidents=[incd for incd, incidentCount in incidentNumList.items() if incidentCount > threshold]
# sumIsolates=len(isolatedIncidents)
# sumDuplicate=len(duplicateIncidents)

In [37]:
len(duplicateIncidentList)

341334

In [38]:
len(isolatedIncidentList)

1405580

In [39]:
'''check random incident_num with non-1s count to see reason for duplicates
    if incident_num and category are the same, then remove duplicates (later)
'''
df[df['IncidntNum']==150098345]

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
10,150098345,LARCENY/THEFT,PETTY THEFT SHOPLIFTING,Sunday,02/01/2015,14:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075,"(37.7690748003847, -122.413354187018)"
11,150098345,DRUG/NARCOTIC,POSSESSION OF METH-AMPHETAMINE,Sunday,02/01/2015,14:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075,"(37.7690748003847, -122.413354187018)"
12,150098345,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,Sunday,02/01/2015,14:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075,"(37.7690748003847, -122.413354187018)"
13,150098345,WARRANTS,WARRANT ARREST,Sunday,02/01/2015,14:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075,"(37.7690748003847, -122.413354187018)"


In [40]:
# check the original dataframe against 'duplicateIncidentList'
# when 'duplicateIncidentList' matches df['IncidntNum'], then check if the category is the same
# if the categories are the same, retain one row and drop the others
# if the categories are different, retain all rows 

df_duplicates=df[df['IncidntNum'].isin(allIncidentNumbers)].sort_values(by=['IncidntNum'])
df_duplicates_cleaned = df_duplicates.drop_duplicates(subset=['Category','IncidntNum'], keep="first")
df_duplicates_cleaned # this is the final dataframe without duplicates

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
493897,3979,WARRANTS,WARRANT ARREST,Thursday,12/09/2004,16:22,BAYVIEW,"ARREST, BOOKED",INGALLS ST / HUDSON AV,-122.379598,37.732467,"(37.7324666541275, -122.379598260097)"
1209080,10128,WARRANTS,WARRANT ARREST,Sunday,12/18/2005,22:20,INGLESIDE,"ARREST, BOOKED",ELLSWORTH ST / ALEMANY BL,-122.418969,37.732208,"(37.7322083102079, -122.4189687006)"
1490233,10736,WARRANTS,WARRANT ARREST,Thursday,07/15/2004,10:18,SOUTHERN,"ARREST, BOOKED",900 Block of BRYANT ST,-122.405927,37.773427,"(37.7734271309539, -122.405926775837)"
1393001,38261,WARRANTS,WARRANT ARREST,Thursday,04/17/2003,22:45,NORTHERN,"ARREST, BOOKED",POLK ST / SUTTER ST,-122.420120,37.787757,"(37.7877570602182, -122.420120319211)"
377806,52205,WARRANTS,WARRANT ARREST,Thursday,02/06/2003,07:20,SOUTHERN,"ARREST, BOOKED",900 Block of BRYANT ST,-122.405927,37.773427,"(37.7734271309539, -122.405926775837)"
1333777,61397,WARRANTS,WARRANT ARREST,Sunday,06/29/2003,10:00,TENDERLOIN,"ARREST, BOOKED",100 Block of TURK ST,-122.411593,37.783053,"(37.7830529510782, -122.411593095704)"
596484,62389,VEHICLE THEFT,STOLEN MOTORCYCLE,Tuesday,03/30/2004,11:35,MISSION,NONE,2600 Block of BRYANT ST,-122.409121,37.751979,"(37.7519787472165, -122.409121399225)"
391902,62953,WARRANTS,WARRANT ARREST,Sunday,08/22/2004,18:36,SOUTHERN,"ARREST, BOOKED",COLINPKELLYJR ST / TOWNSEND ST,-122.389758,37.781221,"(37.781221050395, -122.389757937508)"
1485730,70796,WARRANTS,ENROUTE TO OUTSIDE JURISDICTION,Thursday,07/13/2006,08:45,SOUTHERN,"ARREST, BOOKED",900 Block of MISSION ST,-122.407933,37.781506,"(37.7815063666429, -122.407932868203)"
864778,73057,WARRANTS,WARRANT ARREST,Thursday,08/26/2004,12:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"


In [41]:
df=df_duplicates_cleaned.copy() # final dataframe
print("No. of rows and columns of final dataframe: ",df.shape)

No. of rows and columns of final dataframe:  (2057667, 12)


In [43]:
df[df['IncidntNum']==150098345]

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
13,150098345,WARRANTS,WARRANT ARREST,Sunday,02/01/2015,14:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075,"(37.7690748003847, -122.413354187018)"
12,150098345,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,Sunday,02/01/2015,14:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075,"(37.7690748003847, -122.413354187018)"
10,150098345,LARCENY/THEFT,PETTY THEFT SHOPLIFTING,Sunday,02/01/2015,14:00,MISSION,"ARREST, BOOKED",1700 Block of HARRISON ST,-122.413354,37.769075,"(37.7690748003847, -122.413354187018)"


In [45]:
df.reset_index(drop=True, inplace=True)
df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
0,3979,WARRANTS,WARRANT ARREST,Thursday,12/09/2004,16:22,BAYVIEW,"ARREST, BOOKED",INGALLS ST / HUDSON AV,-122.379598,37.732467,"(37.7324666541275, -122.379598260097)"
1,10128,WARRANTS,WARRANT ARREST,Sunday,12/18/2005,22:20,INGLESIDE,"ARREST, BOOKED",ELLSWORTH ST / ALEMANY BL,-122.418969,37.732208,"(37.7322083102079, -122.4189687006)"
2,10736,WARRANTS,WARRANT ARREST,Thursday,07/15/2004,10:18,SOUTHERN,"ARREST, BOOKED",900 Block of BRYANT ST,-122.405927,37.773427,"(37.7734271309539, -122.405926775837)"
3,38261,WARRANTS,WARRANT ARREST,Thursday,04/17/2003,22:45,NORTHERN,"ARREST, BOOKED",POLK ST / SUTTER ST,-122.420120,37.787757,"(37.7877570602182, -122.420120319211)"
4,52205,WARRANTS,WARRANT ARREST,Thursday,02/06/2003,07:20,SOUTHERN,"ARREST, BOOKED",900 Block of BRYANT ST,-122.405927,37.773427,"(37.7734271309539, -122.405926775837)"


### 2. Check the unique category of crimes
To-dos:
* Group similar crimes together

In [46]:
# check set of crimes
print(len(set(df['Category'])))
set(df['Category'])

39


{'ARSON',
 'ASSAULT',
 'BAD CHECKS',
 'BRIBERY',
 'BURGLARY',
 'DISORDERLY CONDUCT',
 'DRIVING UNDER THE INFLUENCE',
 'DRUG/NARCOTIC',
 'DRUNKENNESS',
 'EMBEZZLEMENT',
 'EXTORTION',
 'FAMILY OFFENSES',
 'FORGERY/COUNTERFEITING',
 'FRAUD',
 'GAMBLING',
 'KIDNAPPING',
 'LARCENY/THEFT',
 'LIQUOR LAWS',
 'LOITERING',
 'MISSING PERSON',
 'NON-CRIMINAL',
 'OTHER OFFENSES',
 'PORNOGRAPHY/OBSCENE MAT',
 'PROSTITUTION',
 'RECOVERED VEHICLE',
 'ROBBERY',
 'RUNAWAY',
 'SECONDARY CODES',
 'SEX OFFENSES, FORCIBLE',
 'SEX OFFENSES, NON FORCIBLE',
 'STOLEN PROPERTY',
 'SUICIDE',
 'SUSPICIOUS OCC',
 'TREA',
 'TRESPASS',
 'VANDALISM',
 'VEHICLE THEFT',
 'WARRANTS',
 'WEAPON LAWS'}

## 1. Reduce number of crime categories

Group the criminal activities into the following categories:
1. theft
2. public disturbance
3. drug
4. sex
5. suicide
6. kidnap
7. domestic
8. other
9. non-criminal (can delete)

In [47]:
theft = ['BRIBERY',
        'BURGLARY',
        'EMBEZZLEMENT',
        'EXTORTION',
        'FORGERY/COUNTERFEITING',
        'FRAUD',
        'GAMBLING',
        'LARCENY/THEFT',
        'RECOVERED VEHICLE',
        'ROBBERY',
        'TREA',
        'VEHICLE THEFT',
        'STOLEN PROPERTY',
        'ARSON']

public = ['TRESPASS',
        'VANDALISM',
        'WARRANTS',
        'LOITERING',
        'ASSAULT',
        'BAD CHECKS',
        'DISORDERLY CONDUCT',
        'DRIVING UNDER THE INFLUENCE',
        'DRUNKENNESS',
        'SUSPICIOUS OCC',
         'LIQUOR LAWS',
         'WEAPON LAWS']

drug =  ['DRUG/NARCOTIC']

sex = ['PORNOGRAPHY/OBSCENE MAT',
        'PROSTITUTION',
        'SEX OFFENSES, FORCIBLE',
        'SEX OFFENSES, NON FORCIBLE']

suicide = ['SUICIDE']

kidnap = ['KIDNAPPING']

domestic = ['MISSING PERSON',
            'RUNAWAY',
            'FAMILY OFFENSES']

others = ['OTHER OFFENSES',
        'SECONDARY CODES']

non_criminal = ['NON-CRIMINAL']

In [48]:
replace_dict = {
                "theft": theft,
                "public": public,
                "drug": drug,
                "sex": sex,
                "suicide": suicide,
                "kidnap":kidnap,
                "domestic": domestic,
                "others": others,
                "non_criminal": non_criminal
               }

In [49]:
new_cat = df['Category']

for key, val in replace_dict.items():
    new_cat = new_cat.replace(val, key)

df["new_cat"] = new_cat

In [50]:
set(df["new_cat"])

{'domestic',
 'drug',
 'kidnap',
 'non_criminal',
 'others',
 'public',
 'sex',
 'suicide',
 'theft'}

In [51]:
df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,new_cat
0,3979,WARRANTS,WARRANT ARREST,Thursday,12/09/2004,16:22,BAYVIEW,"ARREST, BOOKED",INGALLS ST / HUDSON AV,-122.379598,37.732467,"(37.7324666541275, -122.379598260097)",public
1,10128,WARRANTS,WARRANT ARREST,Sunday,12/18/2005,22:20,INGLESIDE,"ARREST, BOOKED",ELLSWORTH ST / ALEMANY BL,-122.418969,37.732208,"(37.7322083102079, -122.4189687006)",public
2,10736,WARRANTS,WARRANT ARREST,Thursday,07/15/2004,10:18,SOUTHERN,"ARREST, BOOKED",900 Block of BRYANT ST,-122.405927,37.773427,"(37.7734271309539, -122.405926775837)",public
3,38261,WARRANTS,WARRANT ARREST,Thursday,04/17/2003,22:45,NORTHERN,"ARREST, BOOKED",POLK ST / SUTTER ST,-122.420120,37.787757,"(37.7877570602182, -122.420120319211)",public
4,52205,WARRANTS,WARRANT ARREST,Thursday,02/06/2003,07:20,SOUTHERN,"ARREST, BOOKED",900 Block of BRYANT ST,-122.405927,37.773427,"(37.7734271309539, -122.405926775837)",public


In [53]:
# drop redundant columns
df.drop(columns=['Category','Descript','Time','Resolution','Address','X','Y','Location'], inplace=True)

In [54]:
df.head()

,IncidntNum,DayOfWeek,Date,PdDistrict,new_cat
0,3979,Thursday,12/09/2004,BAYVIEW,public
1,10128,Sunday,12/18/2005,INGLESIDE,public
2,10736,Thursday,07/15/2004,SOUTHERN,public
3,38261,Thursday,04/17/2003,NORTHERN,public
4,52205,Thursday,02/06/2003,SOUTHERN,public


In [55]:
df.shape

(2057667, 5)

In [56]:
# remove non_criminal
df = df[df['new_cat']!='non_criminal']
df.shape

(1823419, 5)

## 2. Select data between 2012-2017

In [57]:
df['year'] = df['Date'].map(lambda x: pd.to_datetime(x).year)

C:\Users\Wei Tan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
set(df['year'])

{2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018}

In [59]:
df = df[(df['year']>=2012) & (df['year']<=2017)]

In [60]:
set(df['year'])

{2012, 2013, 2014, 2015, 2016, 2017}

## 3. Count number of unique crimes each day

In [61]:
dropdup = df.drop_duplicates(subset='IncidntNum', keep="last")
dropdup.head()

,IncidntNum,DayOfWeek,Date,PdDistrict,new_cat,year
95,1300164,Thursday,02/13/2014,RICHMOND,domestic,2014
214,10073348,Thursday,11/14/2013,TENDERLOIN,others,2013
256,10358128,Thursday,11/14/2013,TENDERLOIN,others,2013
320,10733172,Thursday,03/01/2012,MISSION,public,2012
441,11351210,Tuesday,06/14/2016,BAYVIEW,domestic,2016


In [62]:
dropdup.shape

(611748, 6)

In [63]:
crime_df = dropdup.groupby(['Date', 'DayOfWeek'])['IncidntNum'].agg(['count']).reset_index()
np.sum(crime_df['count'].values)    # sanity check

611748

In [64]:
crime_df.rename(columns={"count": "num_unique_crimes"}, inplace=True)
crime_df.head()

,Date,DayOfWeek,num_unique_crimes
0,01/01/2012,Sunday,358
1,01/01/2013,Tuesday,385
2,01/01/2014,Wednesday,311
3,01/01/2015,Thursday,342
4,01/01/2016,Friday,366


## 4. Count number of each crime types

In [66]:
df.head()

,IncidntNum,DayOfWeek,Date,PdDistrict,new_cat,year
95,1300164,Thursday,02/13/2014,RICHMOND,domestic,2014
214,10073348,Thursday,11/14/2013,TENDERLOIN,others,2013
256,10358128,Thursday,11/14/2013,TENDERLOIN,others,2013
320,10733172,Thursday,03/01/2012,MISSION,public,2012
441,11351210,Tuesday,06/14/2016,BAYVIEW,domestic,2016


In [74]:
dummy_df = pd.get_dummies(df, columns=['new_cat'])

In [75]:
dummy_df.head()

,IncidntNum,DayOfWeek,Date,PdDistrict,year,new_cat_domestic,new_cat_drug,new_cat_kidnap,new_cat_others,new_cat_public,new_cat_sex,new_cat_suicide,new_cat_theft
95,1300164,Thursday,02/13/2014,RICHMOND,2014,1,0,0,0,0,0,0,0
214,10073348,Thursday,11/14/2013,TENDERLOIN,2013,0,0,0,1,0,0,0,0
256,10358128,Thursday,11/14/2013,TENDERLOIN,2013,0,0,0,1,0,0,0,0
320,10733172,Thursday,03/01/2012,MISSION,2012,0,0,0,0,1,0,0,0
441,11351210,Tuesday,06/14/2016,BAYVIEW,2016,1,0,0,0,0,0,0,0


In [76]:
dummy_df.columns

Index(['IncidntNum', 'DayOfWeek', 'Date', 'PdDistrict', 'year',
       'new_cat_domestic', 'new_cat_drug', 'new_cat_kidnap', 'new_cat_others',
       'new_cat_public', 'new_cat_sex', 'new_cat_suicide', 'new_cat_theft'],
      dtype='object')

In [77]:
dummy_df = dummy_df.groupby(['Date', 'DayOfWeek', 'PdDistrict'])['new_cat_domestic', 'new_cat_drug', 'new_cat_kidnap', 'new_cat_others',
       'new_cat_public', 'new_cat_sex', 'new_cat_suicide', 'new_cat_theft'].agg(['sum']).reset_index()

In [80]:
dummy_df.head()

,Date,DayOfWeek,PdDistrict,new_cat_domestic,new_cat_drug,new_cat_kidnap,new_cat_others,new_cat_public,new_cat_sex,new_cat_suicide,new_cat_theft
,,,,sum,sum,sum,sum,sum,sum,sum,sum
0,01/01/2012,Sunday,BAYVIEW,1,1,0,20,17,1,0,9
1,01/01/2012,Sunday,CENTRAL,3,2,0,11,22,0,0,22
2,01/01/2012,Sunday,INGLESIDE,0,0,0,8,11,3,0,15
3,01/01/2012,Sunday,MISSION,2,0,0,7,16,0,0,18
4,01/01/2012,Sunday,NORTHERN,0,1,0,7,14,0,1,10


In [84]:
dummy_df.columns = dummy_df.columns.droplevel(1)

In [85]:
dummy_df.head()

,Date,DayOfWeek,PdDistrict,new_cat_domestic,new_cat_drug,new_cat_kidnap,new_cat_others,new_cat_public,new_cat_sex,new_cat_suicide,new_cat_theft
0,01/01/2012,Sunday,BAYVIEW,1,1,0,20,17,1,0,9
1,01/01/2012,Sunday,CENTRAL,3,2,0,11,22,0,0,22
2,01/01/2012,Sunday,INGLESIDE,0,0,0,8,11,3,0,15
3,01/01/2012,Sunday,MISSION,2,0,0,7,16,0,0,18
4,01/01/2012,Sunday,NORTHERN,0,1,0,7,14,0,1,10


In [87]:
dummy_df['sum_crime_district'] =  dummy_df[['new_cat_domestic', 'new_cat_drug',
                                           'new_cat_kidnap', 'new_cat_others', 'new_cat_public', 'new_cat_sex',
                                           'new_cat_suicide', 'new_cat_theft']].sum(axis=1)

In [93]:
print(dummy_df.shape)
dummy_df.head()

(21920, 12)


,Date,DayOfWeek,PdDistrict,new_cat_domestic,new_cat_drug,new_cat_kidnap,new_cat_others,new_cat_public,new_cat_sex,new_cat_suicide,new_cat_theft,sum_crime_district
0,01/01/2012,Sunday,BAYVIEW,1,1,0,20,17,1,0,9,49
1,01/01/2012,Sunday,CENTRAL,3,2,0,11,22,0,0,22,60
2,01/01/2012,Sunday,INGLESIDE,0,0,0,8,11,3,0,15,37
3,01/01/2012,Sunday,MISSION,2,0,0,7,16,0,0,18,43
4,01/01/2012,Sunday,NORTHERN,0,1,0,7,14,0,1,10,33


In [94]:
# dummy_df.to_csv("crime_types.csv", index=None)

## 5. Combine with sunset sunrise data
* get hours of daylight, sunset and sunrise

In [89]:
sunset_sunrise = pd.read_csv("data/sunrise_sunset.csv")
sunset_sunrise.head()

,date,sunrise_time,sunset_time
0,1/1/2012 0:00,1/1/2012 7:25,1/1/2012 17:01
1,1/2/2012 0:00,1/2/2012 7:25,1/2/2012 17:02
2,1/3/2012 0:00,1/3/2012 7:25,1/3/2012 17:03
3,1/4/2012 0:00,1/4/2012 7:25,1/4/2012 17:04
4,1/5/2012 0:00,1/5/2012 7:25,1/5/2012 17:05


In [100]:
sunset_sunrise['sunrise_time'] = sunset_sunrise['sunrise_time'].apply(lambda x : pd.to_datetime(str(x)))
sunset_sunrise['sunset_time'] = sunset_sunrise['sunset_time'].apply(lambda x : pd.to_datetime(str(x)))

In [118]:
# calculate daylight hours
sunset_sunrise["daylight_hours"] = sunset_sunrise['sunset_time'] - sunset_sunrise['sunrise_time']
sunset_sunrise["daylight_hours"] = sunset_sunrise["daylight_hours"].apply(lambda x: x.seconds/3600)

In [122]:
sunset_sunrise.head()

,date,sunrise_time,sunset_time,daylight_hours
0,1/1/2012 0:00,2012-01-01 07:25:00,2012-01-01 17:01:00,9.600000
1,1/2/2012 0:00,2012-01-02 07:25:00,2012-01-02 17:02:00,9.616667
2,1/3/2012 0:00,2012-01-03 07:25:00,2012-01-03 17:03:00,9.633333
3,1/4/2012 0:00,2012-01-04 07:25:00,2012-01-04 17:04:00,9.650000
4,1/5/2012 0:00,2012-01-05 07:25:00,2012-01-05 17:05:00,9.666667


In [127]:
def convert_time(input_timedate):
    hour = str(input_timedate.hour)
    minute = str(input_timedate.minute)
    if len(minute)==1:
        minute = "0" + str(minute)
    output_time = hour + minute
    return int(output_time)

In [130]:
# reformat sunrise and sunset times
sunset_sunrise['sunrise_time'] = sunset_sunrise['sunrise_time'].apply(lambda x : convert_time(x))
sunset_sunrise['sunset_time'] = sunset_sunrise['sunset_time'].apply(lambda x : convert_time(x))

In [131]:
sunset_sunrise.head()

,date,sunrise_time,sunset_time,daylight_hours
0,1/1/2012 0:00,725,1701,9.600000
1,1/2/2012 0:00,725,1702,9.616667
2,1/3/2012 0:00,725,1703,9.633333
3,1/4/2012 0:00,725,1704,9.650000
4,1/5/2012 0:00,725,1705,9.666667


In [140]:
sunset_sunrise['date'] = sunset_sunrise['date'].apply(lambda x : pd.to_datetime(str(x)))

In [24]:
def convert_date(input_timedate):
    year = str(input_timedate.year)
    month = str(input_timedate.month)
    day = str(input_timedate.day)
    
    if len(month)==1:
        month = "0" + month
    if len(day)==1:
        day = "0" + day
    
    output_date = month + "/" + day + "/" + year
    return output_date

In [145]:
sunset_sunrise['date'] = sunset_sunrise['date'].apply(lambda x : convert_date(x))

In [147]:
sunset_sunrise.rename(columns={"date": "Date"}, inplace=True)

In [148]:
sunset_sunrise.head()

,Date,sunrise_time,sunset_time,daylight_hours
0,01/01/2012,725,1701,9.600000
1,01/02/2012,725,1702,9.616667
2,01/03/2012,725,1703,9.633333
3,01/04/2012,725,1704,9.650000
4,01/05/2012,725,1705,9.666667


## 6. Join both sunrise sunset with crime

In [151]:
dummy_df.shape

(21920, 12)

In [153]:
crime_sun_df = pd.merge(dummy_df, sunset_sunrise, on="Date", how="left")

In [154]:
crime_sun_df.shape

(21920, 15)

In [155]:
crime_sun_df.to_csv("crime_sun_df.csv", index=None)

In [159]:
set(crime_sun_df["PdDistrict"])

{'BAYVIEW',
 'CENTRAL',
 'INGLESIDE',
 'MISSION',
 'NORTHERN',
 'PARK',
 'RICHMOND',
 'SOUTHERN',
 'TARAVAL',
 'TENDERLOIN'}

## 7. Process and merge housing

In [172]:
housing = pd.read_csv('data/HousingPriceFinal.csv')

In [173]:
housing.head()

,Inventory,Median Dom,Median Sale,New Listings,Period End,DoW,Neighborhood,PdDistrict
0,7.0,59.5,1500000,15.0,2/29/2012,Wed,Ashbury Heights / Parnassus,PARK
1,8.0,145.0,293000,7.0,2/29/2012,Wed,Bret Harte,BAYVIEW
2,4.0,69.0,685000,12.0,2/29/2012,Wed,Buena Vista,PARK
3,13.0,72.0,316000,19.0,2/29/2012,Wed,Cathedral Hill,NORTHERN
4,7.0,59.5,552000,10.0,2/29/2012,Wed,Cayuga,INGLESIDE


In [183]:
housing[housing["New Listings"].isnull()]
# means it median sale is not based on new listings?

,Median Dom,Median Sale,New Listings,Date,PdDistrict
1045,39.0,1625000,NaN,01/31/2014,TARAVAL
1068,25.0,2338000,NaN,01/31/2014,RICHMOND
1079,2.0,629000,NaN,02/28/2014,PARK
1113,18.0,3250000,NaN,02/28/2014,RICHMOND
2661,48.5,2375000,NaN,01/31/2017,TARAVAL
2706,43.0,2300000,NaN,02/28/2017,TARAVAL
2713,44.0,2148000,NaN,02/28/2017,RICHMOND
3186,24.0,990000,NaN,12/31/2017,BAYVIEW
3208,18.0,1800000,NaN,01/31/2018,RICHMOND
3962,42.0,805000,NaN,01/31/2018,BAYVIEW


In [175]:
Counter(housing["PdDistrict"])   # sanity check

Counter({'PARK': 932,
         'BAYVIEW': 850,
         'NORTHERN': 765,
         'INGLESIDE': 850,
         'MISSION': 425,
         'CENTRAL': 510,
         'TARAVAL': 1062,
         'RICHMOND': 508,
         'SOUTHERN': 340,
         'TENDERLOIN': 85})

In [176]:
housing.drop(columns=["Inventory", "DoW", "Neighborhood"], inplace=True)

In [177]:
housing.head()

,Median Dom,Median Sale,New Listings,Period End,PdDistrict
0,59.5,1500000,15.0,2/29/2012,PARK
1,145.0,293000,7.0,2/29/2012,BAYVIEW
2,69.0,685000,12.0,2/29/2012,PARK
3,72.0,316000,19.0,2/29/2012,NORTHERN
4,59.5,552000,10.0,2/29/2012,INGLESIDE


In [178]:
housing.rename(columns={"Period End": "Date"}, inplace=True)
housing['Date'] = housing['Date'].apply(lambda x : pd.to_datetime(str(x)))
housing['Date'] = housing['Date'].apply(lambda x : convert_date(x))

In [179]:
housing.head()

,Median Dom,Median Sale,New Listings,Date,PdDistrict
0,59.5,1500000,15.0,02/29/2012,PARK
1,145.0,293000,7.0,02/29/2012,BAYVIEW
2,69.0,685000,12.0,02/29/2012,PARK
3,72.0,316000,19.0,02/29/2012,NORTHERN
4,59.5,552000,10.0,02/29/2012,INGLESIDE


In [196]:
# sanity check for cell below
housing[(housing["PdDistrict"]=="BAYVIEW") & (housing["Date"]=="01/31/2013")]["New Listings"].sum()

505.0

In [197]:
# Housing listings - Nan did not affect sum
housing_sumlistings = housing.groupby(['Date', 'PdDistrict'])['New Listings'].agg(['sum']).reset_index()
housing_sumlistings.rename(columns={"sum": "house_listings_sum"}, inplace=True)

In [214]:
housing_sumlistings.head()

,Date,PdDistrict,house_listings_sum
0,01/31/2013,BAYVIEW,505.0
1,01/31/2013,CENTRAL,131.0
2,01/31/2013,INGLESIDE,130.0
3,01/31/2013,MISSION,137.0
4,01/31/2013,NORTHERN,147.0


In [199]:
housing_sumlistings.isnull().sum()

Date                  0
PdDistrict            0
house_listings_sum    0
dtype: int64

In [206]:
housing_sales = housing.groupby(['Date', 'PdDistrict'])[['Median Dom', 'Median Sale']].agg(['median', 'mean', 'min', 'max']).reset_index()

In [207]:
housing_sales.isnull().sum()

Date                   0
PdDistrict             0
Median Dom   median    0
             mean      0
             min       0
             max       0
Median Sale  median    0
             mean      0
             min       0
             max       0
dtype: int64

In [211]:
# housing_sales.head()

In [209]:
housing_sales.columns = ['Date', 'PdDistrict', 'Dom_median', 'Dom_mean', 'Dom_min', 'Dom_max',
                        'Sale_median', 'Sale_mean', 'Sale_min', 'Sale_max']

In [210]:
housing_sales.head()

,Date,PdDistrict,Dom_median,Dom_mean,Dom_min,Dom_max,Sale_median,Sale_mean,Sale_min,Sale_max
0,01/31/2013,BAYVIEW,21.25,24.600000,13.0,46.0,546000,6.141000e+05,368000,1275000
1,01/31/2013,CENTRAL,17.75,19.000000,11.0,32.0,894000,8.921667e+05,475000,1300000
2,01/31/2013,INGLESIDE,16.00,18.050000,12.5,30.5,583000,6.355000e+05,533000,887000
3,01/31/2013,MISSION,16.00,17.700000,12.0,28.5,1000000,1.036800e+06,804000,1365000
4,01/31/2013,NORTHERN,19.50,20.277778,12.0,33.0,1183000,1.099778e+06,543000,1435000


In [213]:
crime_sun_df.head()

,Date,DayOfWeek,PdDistrict,new_cat_domestic,new_cat_drug,new_cat_kidnap,new_cat_others,new_cat_public,new_cat_sex,new_cat_suicide,new_cat_theft,sum_crime_district,sunrise_time,sunset_time,daylight_hours
0,01/01/2012,Sunday,BAYVIEW,1,1,0,20,17,1,0,9,49,725,1701,9.6
1,01/01/2012,Sunday,CENTRAL,3,2,0,11,22,0,0,22,60,725,1701,9.6
2,01/01/2012,Sunday,INGLESIDE,0,0,0,8,11,3,0,15,37,725,1701,9.6
3,01/01/2012,Sunday,MISSION,2,0,0,7,16,0,0,18,43,725,1701,9.6
4,01/01/2012,Sunday,NORTHERN,0,1,0,7,14,0,1,10,33,725,1701,9.6


In [216]:
housing_sumlistings[(housing_sumlistings["Date"]=="01/01/2012") & (housing_sumlistings["PdDistrict"]=="BAYVIEW")]

,Date,PdDistrict,house_listings_sum


In [217]:
crime_sun_house_df = pd.merge(crime_sun_df, housing_sumlistings, on=["Date", "PdDistrict"], how="left")
crime_sun_house_df = pd.merge(crime_sun_house_df, housing_sales, on=["Date", "PdDistrict"], how="left")

In [219]:
crime_sun_house_df.head()

,Date,DayOfWeek,PdDistrict,new_cat_domestic,new_cat_drug,new_cat_kidnap,new_cat_others,new_cat_public,new_cat_sex,new_cat_suicide,...,daylight_hours,house_listings_sum,Dom_median,Dom_mean,Dom_min,Dom_max,Sale_median,Sale_mean,Sale_min,Sale_max
0,01/01/2012,Sunday,BAYVIEW,1,1,0,20,17,1,0,...,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2012,Sunday,CENTRAL,3,2,0,11,22,0,0,...,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2012,Sunday,INGLESIDE,0,0,0,8,11,3,0,...,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2012,Sunday,MISSION,2,0,0,7,16,0,0,...,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2012,Sunday,NORTHERN,0,1,0,7,14,0,1,...,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
crime_sun_house_df.isnull().sum()

Date                      0
DayOfWeek                 0
PdDistrict                0
new_cat_domestic          0
new_cat_drug              0
new_cat_kidnap            0
new_cat_others            0
new_cat_public            0
new_cat_sex               0
new_cat_suicide           0
new_cat_theft             0
sum_crime_district        0
sunrise_time              0
sunset_time               0
daylight_hours            0
house_listings_sum    21210
Dom_median            21210
Dom_mean              21210
Dom_min               21210
Dom_max               21210
Sale_median           21210
Sale_mean             21210
Sale_min              21210
Sale_max              21210
dtype: int64

In [221]:
crime_sun_house_df.to_csv("crime_sun_house.csv", index=None)

## 8. Process and merge eviction data

In [17]:
eviction = pd.read_csv('data/eviction_minus.csv')
df = pd.read_csv('data/crime_sun_house.csv')

In [18]:
eviction.head()

,Eviction.ID,Address,City,State,Eviction.Notice.Source.Zipcode,File.Date,Non.Payment,Breach,Nuisance,Illegal.Use,...,Other.Cause,Late.Payments,Lead.Remediation,Development,Good.Samaritan.Ends,Constraints.Date,Supervisor.District,Neighborhoods...Analysis.Boundaries,Location,PdDistrict
0,M190330,1800 Block Of Market Street,San Francisco,CA,94102.0,2/15/2019,False,False,True,False,...,False,False,False,False,False,NaN,8.0,Hayes Valley,"(37.7712923782185, -122.42433694872517)",SOUTHERN
1,M183362,1100 Block Of Mission Street,San Francisco,CA,94103.0,12/20/2018,False,False,True,False,...,False,False,False,False,False,NaN,6.0,South of Market,"(37.77779030910167, -122.41283433258292)",SOUTHERN
2,M183157,0 Block Of Brady Street,San Francisco,CA,94103.0,11/21/2018,False,True,False,False,...,False,False,False,False,False,NaN,6.0,Mission,"(37.772548429191886, -122.42008435391327)",SOUTHERN
3,M182998,0 Block Of Stillman Street,San Francisco,CA,94107.0,10/29/2018,False,False,True,False,...,False,False,False,False,False,NaN,6.0,Financial District/South Beach,"(37.78258788740649, -122.3952163164336)",SOUTHERN
4,M182997,0 Block Of Stillman Street,San Francisco,CA,94107.0,10/29/2018,False,True,False,False,...,False,False,False,False,False,NaN,6.0,Financial District/South Beach,"(37.78258788740649, -122.3952163164336)",SOUTHERN


In [28]:
eviction['File.Date'] = eviction['File.Date'].apply(lambda x : pd.to_datetime(str(x)))

In [25]:
eviction['File.Date'] = eviction['File.Date'].apply(lambda x : convert_date(x))

In [99]:
eviction.rename(columns={"File.Date": "Date"}, inplace=True)
eviction.head()

,Eviction.ID,Address,City,State,Eviction.Notice.Source.Zipcode,Date,Non.Payment,Breach,Nuisance,Illegal.Use,Failure.to.Sign.Renewal,Access.Denial,Unapproved.Subtenant,Owner.Move.In,Demolition,Capital.Improvement,Substantial.Rehab,Ellis.Act.WithDrawal,Condo.Conversion,Roommate.Same.Unit,Other.Cause,Late.Payments,Lead.Remediation,Development,Good.Samaritan.Ends,Constraints.Date,Supervisor.District,Neighborhoods...Analysis.Boundaries,Location,PdDistrict,month,year
0,M190330,1800 Block Of Market Street,San Francisco,CA,94102.0,02/15/2019,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,8.0,Hayes Valley,"(37.7712923782185, -122.42433694872517)",SOUTHERN,2,2019
1,M183362,1100 Block Of Mission Street,San Francisco,CA,94103.0,12/20/2018,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,6.0,South of Market,"(37.77779030910167, -122.41283433258292)",SOUTHERN,12,2018
2,M183157,0 Block Of Brady Street,San Francisco,CA,94103.0,11/21/2018,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,6.0,Mission,"(37.772548429191886, -122.42008435391327)",SOUTHERN,11,2018
3,M182998,0 Block Of Stillman Street,San Francisco,CA,94107.0,10/29/2018,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,6.0,Financial District/South Beach,"(37.78258788740649, -122.3952163164336)",SOUTHERN,10,2018
4,M182997,0 Block Of Stillman Street,San Francisco,CA,94107.0,10/29/2018,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,6.0,Financial District/South Beach,"(37.78258788740649, -122.3952163164336)",SOUTHERN,10,2018


In [100]:
eviction.shape

(40091, 32)

In [93]:
eviction['PdDistrict'].unique()

array(['SOUTHERN', 'BAYVIEW', 'MISSION', 'NORTHERN', 'TENDERLOIN',
       'CENTRAL', 'PARK', 'RICHMOND', 'INGLESIDE', 'TARAVAL', nan],
      dtype=object)

In [103]:
eviction_dailysum = eviction.groupby(['Date', 'PdDistrict'])['Eviction.ID'].count().reset_index()
eviction_dailysum.rename(columns={"Eviction.ID": "EvictionNos"}, inplace=True)

In [104]:
eviction_dailysum.head()

,Date,PdDistrict,EvictionNos
0,01/01/2011,PARK,1
1,01/02/1997,CENTRAL,1
2,01/02/1997,INGLESIDE,2
3,01/02/1997,MISSION,2
4,01/02/1997,NORTHERN,1


In [106]:
eviction_dailysum['PdDistrict'].unique()

array(['PARK', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'RICHMOND',
       'SOUTHERN', 'TARAVAL', 'BAYVIEW', 'TENDERLOIN'], dtype=object)

In [107]:
eviction_dailysum['EvictionNos'].unique()

array([  1,   2,   3,   4,   7,   5,  11,   6,   9,   8,  16,  27,  12,
        10, 230,  23,  14,  15,  13,  48,  38,  24,  21,  25,  34,  18,
       235,  36,  31,  28,  44, 111,  39,  19,  20])

In [116]:
Counter(eviction_dailysum['EvictionNos'])

Counter({1: 13208,
         2: 4918,
         3: 1995,
         4: 830,
         7: 129,
         5: 335,
         11: 13,
         6: 204,
         9: 37,
         8: 65,
         16: 2,
         27: 3,
         12: 10,
         10: 26,
         230: 1,
         23: 3,
         14: 5,
         15: 9,
         13: 9,
         48: 1,
         38: 1,
         24: 2,
         21: 3,
         25: 1,
         34: 3,
         18: 2,
         235: 1,
         36: 1,
         31: 2,
         28: 1,
         44: 1,
         111: 1,
         39: 1,
         19: 2,
         20: 1})

In [52]:
# if we want to count by month
#eviction['month'] = eviction['Date'].map(lambda x: pd.to_datetime(x).month)
#eviction['year'] = eviction['Date'].map(lambda x: pd.to_datetime(x).year)

In [60]:
# eviction_monthsum = eviction.groupby(['year', 'month', 'PdDistrict'])['Eviction.ID'].count().reset_index()
# eviction_monthsum.rename(columns={"Eviction.ID": "EvictionNos"}, inplace=True)

In [108]:
#eviction_monthsum['EvictionNos'].nunique()
#Counter(eviction_monthsum['EvictionNos'])

Merge eviction dataset with existing combined dataset (crime, house, sunset/sunrise times) 

In [109]:
df.head()

,Date,DayOfWeek,PdDistrict,new_cat_domestic,new_cat_drug,new_cat_kidnap,new_cat_others,new_cat_public,new_cat_sex,new_cat_suicide,new_cat_theft,sum_crime_district,sunrise_time,sunset_time,daylight_hours,house_listings_sum,Dom_median,Dom_mean,Dom_min,Dom_max,Sale_median,Sale_mean,Sale_min,Sale_max
0,01/01/2012,Sunday,BAYVIEW,1,1,0,20,17,1,0,9,49,725,1701,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2012,Sunday,CENTRAL,3,2,0,11,22,0,0,22,60,725,1701,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2012,Sunday,INGLESIDE,0,0,0,8,11,3,0,15,37,725,1701,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2012,Sunday,MISSION,2,0,0,7,16,0,0,18,43,725,1701,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2012,Sunday,NORTHERN,0,1,0,7,14,0,1,10,33,725,1701,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
crime_sun_house_eviction=pd.merge(df,eviction_dailysum,on=["Date", "PdDistrict"],how="left")

In [111]:
crime_sun_house_eviction

,Date,DayOfWeek,PdDistrict,new_cat_domestic,new_cat_drug,new_cat_kidnap,new_cat_others,new_cat_public,new_cat_sex,new_cat_suicide,new_cat_theft,sum_crime_district,sunrise_time,sunset_time,daylight_hours,house_listings_sum,Dom_median,Dom_mean,Dom_min,Dom_max,Sale_median,Sale_mean,Sale_min,Sale_max,EvictionNos
0,01/01/2012,Sunday,BAYVIEW,1,1,0,20,17,1,0,9,49,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2012,Sunday,CENTRAL,3,2,0,11,22,0,0,22,60,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2012,Sunday,INGLESIDE,0,0,0,8,11,3,0,15,37,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2012,Sunday,MISSION,2,0,0,7,16,0,0,18,43,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2012,Sunday,NORTHERN,0,1,0,7,14,0,1,10,33,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,01/01/2012,Sunday,PARK,0,0,0,5,6,0,1,4,16,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,01/01/2012,Sunday,RICHMOND,4,0,0,11,10,0,0,7,32,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,01/01/2012,Sunday,SOUTHERN,6,4,0,26,26,0,1,30,93,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,01/01/2012,Sunday,TARAVAL,1,0,0,12,11,2,0,12,38,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,01/01/2012,Sunday,TENDERLOIN,1,3,1,3,5,1,0,5,19,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
crime_sun_house_eviction['EvictionNos']=crime_sun_house_eviction['EvictionNos'].fillna(0)

In [115]:
crime_sun_house_eviction

,Date,DayOfWeek,PdDistrict,new_cat_domestic,new_cat_drug,new_cat_kidnap,new_cat_others,new_cat_public,new_cat_sex,new_cat_suicide,new_cat_theft,sum_crime_district,sunrise_time,sunset_time,daylight_hours,house_listings_sum,Dom_median,Dom_mean,Dom_min,Dom_max,Sale_median,Sale_mean,Sale_min,Sale_max,EvictionNos
0,01/01/2012,Sunday,BAYVIEW,1,1,0,20,17,1,0,9,49,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,01/01/2012,Sunday,CENTRAL,3,2,0,11,22,0,0,22,60,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,01/01/2012,Sunday,INGLESIDE,0,0,0,8,11,3,0,15,37,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,01/01/2012,Sunday,MISSION,2,0,0,7,16,0,0,18,43,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,01/01/2012,Sunday,NORTHERN,0,1,0,7,14,0,1,10,33,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,01/01/2012,Sunday,PARK,0,0,0,5,6,0,1,4,16,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,01/01/2012,Sunday,RICHMOND,4,0,0,11,10,0,0,7,32,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,01/01/2012,Sunday,SOUTHERN,6,4,0,26,26,0,1,30,93,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8,01/01/2012,Sunday,TARAVAL,1,0,0,12,11,2,0,12,38,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,01/01/2012,Sunday,TENDERLOIN,1,3,1,3,5,1,0,5,19,725,1701,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [117]:
crime_sun_house_eviction.to_csv("crime_sun_house_eviction.csv", index=None)